# Installation

In [ ]:
# %pip install torch
# %pip install torchtext
# %pip install conllu

In [ ]:
# !unzip data.zip

# import

In [ ]:
import torch
torch.manual_seed(42) # Set Seed for reproducibility
import conllu
from io import open
from torch.utils.data import Dataset
from torchtext.vocab import build_vocab_from_iterator, Vocab
from torch.nn.utils.rnn import pad_sequence
from torch.utils.data import DataLoader
import csv
import json
# from tainEmbeddings import pretrainedEmbeddings

# Get Data

In [ ]:
# Read the CoNLL-U file
def read_conllu_file(file_path):
    with open(file_path, "r", encoding="utf-8") as f:
        data = f.read()
    return data

# Parse the CoNLL-U data
def parse_conllu_data(conllu_data):
    return conllu.parse(conllu_data)

# Extract sentences and POS tags
def extract_sentences_and_postags(parsed_data):
    sentences_and_postags = []
    for tokenList in parsed_data:
        sentence = []
        postagList = []
        for token in tokenList:
            # print(token)
            word = token['form']
            postag = token['upostag']
            # filter out punctuation
            if postag != "PUNCT":
                sentence.append(word)
                postagList.append(postag)
        # if len(sentence) == 0 or len(postagList) == 0:
        #     print("Empty sentence or postagList")
        sentences_and_postags.append((sentence, postagList))
    return sentences_and_postags

def getSentencesAndPosTags(conllu_file_path):
    conllu_data = read_conllu_file(conllu_file_path)
    parsed_data = parse_conllu_data(conllu_data)
    # print(parsed_data)
    return extract_sentences_and_postags(parsed_data)

# sentences_and_postags = getSentencesAndPosTags("./data/UD_English-Atis/en_atis-ud-train.conllu")
# # Print the list of words and POS tags
# for sentence, postagList in sentences_and_postags:
#     print(f"sentence: {sentence}, postagList: {postagList}")


# Dataset

In [ ]:
START_TOKEN = "<s>"
END_TOKEN = "</s>"
UNKNOWN_TOKEN = "<unk>"
PAD_TOKEN = "<pad>"
SPECIAL_TOKENS = [START_TOKEN, END_TOKEN, UNKNOWN_TOKEN, PAD_TOKEN]
SPECIAL_POS_TAG = "SPECIAL"

class PosDataset(Dataset):
    def __init__(self, data : list[tuple[list[str], list[str]]] , pOfNN: int , sOfNN: int, wordVocabulary:Vocab|None=None , tagVocabulary:Vocab|None=None):
        """Initialize the dataset. Setup Code goes here"""
        self.pOfNN = pOfNN
        self.sOfNN = sOfNN
        # self.sentences = [i[0] for i in data] # list of sentences
        self.chunks = []
        for i in range(len(data)):
            sentence = self.insertStartEndTokens(data[i][0])
            chunksList = [sentence[i: i+pOfNN+sOfNN+1] for i in range(len(sentence)-pOfNN-sOfNN)]
            self.chunks.extend(chunksList)
        self.labels = []
        for i in range(len(data)):
            sentenceTags = data[i][1]
            self.labels.extend(sentenceTags)

        if wordVocabulary is None:
            sentences = [i[0] for i in data] # list of sentences
            self.wordVocabulary = build_vocab_from_iterator(sentences, specials= SPECIAL_TOKENS, min_freq= 2) # use min_freq for handling unkown words better
            self.wordVocabulary.set_default_index(self.wordVocabulary[UNKNOWN_TOKEN])
        else:
            # if vocabulary provided use that
            self.wordVocabulary = wordVocabulary
        if tagVocabulary is None:
            self.tagVocabulary = build_vocab_from_iterator([self.labels], specials= [SPECIAL_POS_TAG])
            self.tagVocabulary.set_default_index(self.tagVocabulary[SPECIAL_POS_TAG]) # index for unknown tags
        else:
            # if vocabulary provided use that
            self.tagVocabulary = tagVocabulary


    def __len__(self) -> int:
        """Returns number of datapoints."""
        return len(self.chunks)

    def __getitem__(self, index: int) -> tuple[torch.Tensor, torch.Tensor]:
        """Get the datapoint at `index`."""
        posTagIndex = self.tagVocabulary[self.labels[index]]
        # # create a one hot vector for the pos tag
        # posTagVector = torch.zeros(len(self.tagVocabulary))
        # posTagVector[posTagIndex] = 1
        posTagVector = posTagIndex # not converting to vector
        return torch.tensor(self.wordVocabulary.lookup_indices(self.chunks[index])), torch.tensor(posTagVector)

    def insertStartEndTokens(self, sentence: list[str]) -> list[str]:
        """Add start and end tokens to the sentence"""
        return [START_TOKEN]*self.pOfNN + sentence + [END_TOKEN]*self.sOfNN

    def createCSV(self, filename: str):
        # create a csv file with the chunks and labels to test if the dataset is created correctly
        with open(filename, 'w', newline='') as file:
            writer = csv.writer(file)
            writer.writerow(["chunks", "labels"])
            for i in range(len(self.chunks)):
                writer.writerow([self.chunks[i], self.labels[i]])

    def collate(self, batch: list[tuple[torch.Tensor, torch.Tensor]]) -> tuple[torch.Tensor, torch.Tensor]:
        # """Given a list of datapoints, batch them together"""
        # sentences = [i[0] for i in batch]
        # labels = [i[1] for i in batch]
        # padded_sentences = pad_sequence(sentences, batch_first=True, padding_value=self.vocabulary[PAD_TOKEN]) # pad sentences with pad token id
        # padded_labels = pad_sequence(sentences, batch_first=True, padding_value=torch.tensor(0)) # pad labels with 0 because pad token cannot be entities
        # return padded_sentences, padded_labels
        pass

# Fast Forward Module

In [ ]:
class FFN(torch.nn.Module):
    def __init__(self, vocabularySize: int, embeddingSize: int, p: int, s: int, tagVocabSize: int, activationFunction: torch.nn.Module, hiddenLayerSizes: list[int] = []):
        super(FFN, self).__init__()
        self.embeddingModule = torch.nn.Embedding(
            vocabularySize, embeddingSize)
        # self.embeddingModule = torch.nn.Embedding.from_pretrained(pretrainedEmbeddings)
        # self.embeddingModule.weight.requires_grad = False
        self.p = p
        self.s = s
        self.vocabularySize = vocabularySize
        self.embeddingSize = embeddingSize
        self.tagVocabSize = tagVocabSize
        self.activationFunction = activationFunction
        self.numOfInputWords = p + s + 1
        self.inputSize = embeddingSize*self.numOfInputWords
        self.outputSize = tagVocabSize
        self.hiddenLayerSizes = hiddenLayerSizes
        moduleList = []
        for i in range(len(self.hiddenLayerSizes)):
            if i == 0:
                moduleList.append(torch.nn.Linear(
                    self.inputSize, self.hiddenLayerSizes[i]))
            else:
                moduleList.append(torch.nn.Linear(
                    self.hiddenLayerSizes[i-1], self.hiddenLayerSizes[i]))
            moduleList.append(activationFunction)
        moduleList.append(torch.nn.Linear(
            self.hiddenLayerSizes[-1], self.outputSize))
        self.posTagger = torch.nn.Sequential(*moduleList)

    # wordIndices is a tensor of indices of words in the chunk
    def forward(self, wordIndices: torch.tensor):
        wordEmbeddings = self.embeddingModule(wordIndices)
        wordEmbeddings = wordEmbeddings.view(wordEmbeddings.size(0), -1)
        return self.posTagger(wordEmbeddings)


# HyperParameters

In [ ]:
hyperParameters0 = {
    'trainBatchSize': 64,
    'evalBatchSize': 64,
    'testBatchSize': 64,
    'p': 2,
    's': 3,
    'learningRate': 1e-2,
    'numEpochs': 10,
    'activationFunction': torch.nn.ReLU(),
    'hiddenDimList': [512,200],
    'embeddingDim': 300,
    'modelPath' : "ffnDiffConf2.pt"
}
hyperParameters1 = {
    'trainBatchSize': 64,
    'evalBatchSize': 64,
    'testBatchSize': 64,
    'p': 2,
    's': 3,
    'learningRate': 1e-1,
    'numEpochs': 10,
    'activationFunction': torch.nn.Tanh(),
    'hiddenDimList': [64,32,16],
    'embeddingDim': 64,
    'modelPath' : "ffnDiffConf3.pt"
}
hyperParameterList = [hyperParameters0, hyperParameters1]
hyperParameters = None
trainRawData = getSentencesAndPosTags("./data/UD_English-Atis/en_atis-ud-train.conllu")
evalRawData = getSentencesAndPosTags("./data/UD_English-Atis/en_atis-ud-dev.conllu")
testRawData = getSentencesAndPosTags("./data/UD_English-Atis/en_atis-ud-test.conllu")
for config in hyperParameterList:
  hyperParameters = config
  trainDataset = PosDataset(trainRawData , pOfNN= hyperParameters['p'], sOfNN= hyperParameters['s'])
  evalDataset = PosDataset(evalRawData ,pOfNN= hyperParameters['p'], sOfNN= hyperParameters['s'], wordVocabulary=trainDataset.wordVocabulary, tagVocabulary=trainDataset.tagVocabulary)
  testDataset = PosDataset(testRawData ,pOfNN= hyperParameters['p'], sOfNN= hyperParameters['s'], wordVocabulary=trainDataset.wordVocabulary, tagVocabulary=trainDataset.tagVocabulary)
  trainDataLoader = DataLoader(trainDataset, batch_size=hyperParameters['trainBatchSize'], shuffle=True)
  evalDataLoader = DataLoader(evalDataset, batch_size=hyperParameters['evalBatchSize']) # dont shuffle here for consistent evaluation
  testDataLoader = DataLoader(testDataset, batch_size=hyperParameters['testBatchSize'])

  # # create train.csv, eval.csv, test.csv
  # trainDataset.createCSV("train.csv")
  # evalDataset.createCSV("eval.csv")
  # testDataset.createCSV("test.csv")
  # # store vocabularies
  # wordVocabPath = 'trainWordVocab.pth'
  # torch.save(trainDataset.wordVocabulary, wordVocabPath)
  # tagVocabPath = 'trainTagVocab.pth'
  # torch.save(trainDataset.tagVocabulary, tagVocabPath)

  print(trainDataset[7])
  trainWordVocabSize = len(trainDataset.wordVocabulary)
  trainTagVocabSize = len(trainDataset.tagVocabulary)
  myPosTagger = FFN (trainWordVocabSize,hyperParameters['embeddingDim'],hyperParameters['p'],hyperParameters['s'], trainTagVocabSize, hyperParameters['activationFunction'], hyperParameters['hiddenDimList'])
  device = "cuda" if torch.cuda.is_available() else "cpu"
  print(device)
  lossFn = torch.nn.CrossEntropyLoss()
  optimizer = torch.optim.Adam(myPosTagger.parameters(), lr=hyperParameters['learningRate'])


  # Send Model to  Device
  myPosTagger = myPosTagger.to(device)

  for epochNum in range(hyperParameters['numEpochs']):
      # Set to Training Mode
      myPosTagger.train()
      for batchNum, (inputChunks , tags) in enumerate(trainDataLoader):
          # Zero the gradients
          optimizer.zero_grad()
          # Send data to device
          inputChunks = inputChunks.to(device)
          tags = tags.to(device)
          # Forward Pass
          pred = myPosTagger(inputChunks)
          loss = lossFn(pred, tags)
          # Backward Pass
          loss.backward()
          optimizer.step()
          # Print Loss after every 50 batches
          if batchNum % 50 == 0:
              print(f'Epoch: {epochNum}, Batch: {batchNum}, Loss: {loss.item()}')

      # Set to Evaluation Mode
      myPosTagger.eval()
      evalLoss = 0
      correct = 0
      total = 0
      # turn off gradients
      with torch.no_grad():
          for batchNum, (inputChunks , tags) in enumerate(evalDataLoader):
              # Send data to device
              inputChunks = inputChunks.to(device)
              tags = tags.to(device)
              # Forward Pass
              pred = myPosTagger(inputChunks)
              loss = lossFn(pred, tags)
              evalLoss += loss.item()
              # Get Predictions
              pred = torch.argmax(pred, dim=1)
              # tagIndices = torch.argmax(tags, dim=1)
              tagIndices = tags
              total += tags.size(0)
              correct += (pred == tagIndices).sum().item()
      # Print validation Loss and Accuracy
      print(f'Epoch: {epochNum}, validation Loss: {evalLoss/len(evalDataLoader)}, Val Accuracy: {(correct/total)*100} %')


  torch.save(myPosTagger,hyperParameters['modelPath'])
  loadedPosTagger = torch.load(hyperParameters['modelPath'])
  # move the model to the GPU
  loadedPosTagger = loadedPosTagger.to(device)
  # set the testing model to the evaluation mode
  loadedPosTagger.eval()
  lossFn = torch.nn.CrossEntropyLoss()
  testLoss = 0
  correct = 0
  total = 0
  # turn off gradients
  with torch.no_grad():
      for batchNum, (inputChunks , tags) in enumerate(testDataLoader):
          # Send data to device
          inputChunks = inputChunks.to(device)
          tags = tags.to(device)
          # Forward Pass
          pred = loadedPosTagger(inputChunks)
          loss = lossFn(pred, tags)
          testLoss += loss.item()
          # Get Predictions
          pred = torch.argmax(pred, dim=1)
          # tagIndices = torch.argmax(tags, dim=1)
          tagIndices = tags
          total += tags.size(0)
          correct += (pred == tagIndices).sum().item()
  # Print test Loss and Accuracy
  print(f'test Loss: {testLoss/len(testDataLoader)}, Test Accuracy: {(correct/total)*100} %')
